# Spectranet Analysis

**`Goal:`** 

1. Conducted sentiment analysis and aspect-based sentiment analysis on extracted Spectranet data using the best model from the sentiment analysis and aspect-based sentiment analysis modeling phases.

- **Best model (Sentiment Analysis):** BERTweet
- **Best model (ABSA):** Binary relevance BERTweets

2. Prepare the data for entry into Tableau for dashboard visualization

## 1. Library importation

In [1]:
import sys
import numpy as np
import pandas as pd

## 2. Load the data

In [2]:
#Load the full dataset
full_isp_data = pd.read_csv("../data/processed/full_merge_cleaned.csv")

#Get the Spectranet ISP tweets – LOL. I mispelled
spectranet_data = full_isp_data.query(" ISP_Name == 'sprectranet' ").dropna(subset=['Text'])

spectranet_data.head()

,ISP_Name,Time,Text,Source
0,sprectranet,2019-05-13 09:30:03,it gives me joy seeing my spectranet turning g...,Twitter for iPhone
1,sprectranet,2020-04-21 06:11:55+00:00,spectranet_ng is this even fair? i won't renew...,Twitter for iPhone
2,sprectranet,2020-02-04 18:30:35+00:00,my family used my spectranet and they don't wa...,Twitter for Android
3,sprectranet,2019-02-16 18:11:48,spectranet_ng can i subscribe via ubagroup mob...,Twitter for Android
4,sprectranet,2020-08-14 06:25:29+00:00,eniolashitta youtube is where spectranet start...,Twitter for Android


## 3. Load the models

In [9]:
#Load the sentiment analysis model
sys.path.append("../models/sentiment_analysis_models")
import bertweet_sentiment_model

In [5]:
#Load the absa model
sys.path.append("../models/full_absa_models")
import binary_relevance_model

2022-04-01 23:01:30.834635: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some layers from the model checkpoint at absa/classifier-rest-0.2 were not used when initializing BertABSClassifier: ['dropout_379']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAB

## 4. Run a sentiment analysis on the tweets

In [10]:
sentiment_results = bertweet_sentiment_model.run(spectranet_data,'Text')

In [11]:
#Append the results to the original tweets
spec_data_with_sentiment = spectranet_data.copy().reset_index(drop=True)
spec_data_with_sentiment[['Predicted sentiment']] = sentiment_results

In [12]:
spec_data_with_sentiment

,ISP_Name,Time,Text,Source,Predicted sentiment
0,sprectranet,2019-05-13 09:30:03,it gives me joy seeing my spectranet turning g...,Twitter for iPhone,Positive
1,sprectranet,2020-04-21 06:11:55+00:00,spectranet_ng is this even fair? i won't renew...,Twitter for iPhone,Negative
2,sprectranet,2020-02-04 18:30:35+00:00,my family used my spectranet and they don't wa...,Twitter for Android,Negative
3,sprectranet,2019-02-16 18:11:48,spectranet_ng can i subscribe via ubagroup mob...,Twitter for Android,Neutral
4,sprectranet,2020-08-14 06:25:29+00:00,eniolashitta youtube is where spectranet start...,Twitter for Android,Neutral
...,...,...,...,...,...
1172,sprectranet,2019-01-27 07:45:17,riqueza_cakes get spectranet then .,Twitter for iPhone,Neutral
1173,sprectranet,2019-03-02 15:08:26,i can't find my spectranet_ng mifi and i still...,Twitter for iPhone,Negative
1174,sprectranet,2020-10-30 00:19:29+00:00,spectranet is always terrible at night. fix up...,Twitter for iPhone,Negative
1175,sprectranet,2020-04-26 07:15:37+00:00,spectranet_ng are we getting 100% today ?,Twitter for iPhone,Neutral


## 5. Run Aspect-based Sentiment Analysis

In [7]:
absa_results = binary_relevance_model.run(spectranet_data,'Text')

In [13]:
absa_results.head()

,Text,Detected aspects,Predicted sentiment
0,it gives me joy seeing my spectranet turning g...,[speed],[Negative]
1,spectranet_ng is this even fair? i won't renew...,[None],[None]
2,my family used my spectranet and they don't wa...,[None],[None]
3,spectranet_ng can i subscribe via ubagroup mob...,[None],[None]
4,eniolashitta youtube is where spectranet start...,[speed],[Negative]


In [14]:
#Append the ABSA results to the dataframe
spec_data_with_sentiment[['Detected aspects',
                          'Aspect sentiment']] = absa_results.iloc[:,1:]

In [15]:
spec_data_with_sentiment

,ISP_Name,Time,Text,Source,Predicted sentiment,Detected aspects,Aspect sentiment
0,sprectranet,2019-05-13 09:30:03,it gives me joy seeing my spectranet turning g...,Twitter for iPhone,Positive,[speed],[Negative]
1,sprectranet,2020-04-21 06:11:55+00:00,spectranet_ng is this even fair? i won't renew...,Twitter for iPhone,Negative,[None],[None]
2,sprectranet,2020-02-04 18:30:35+00:00,my family used my spectranet and they don't wa...,Twitter for Android,Negative,[None],[None]
3,sprectranet,2019-02-16 18:11:48,spectranet_ng can i subscribe via ubagroup mob...,Twitter for Android,Neutral,[None],[None]
4,sprectranet,2020-08-14 06:25:29+00:00,eniolashitta youtube is where spectranet start...,Twitter for Android,Neutral,[speed],[Negative]
...,...,...,...,...,...,...,...
1172,sprectranet,2019-01-27 07:45:17,riqueza_cakes get spectranet then .,Twitter for iPhone,Neutral,[None],[None]
1173,sprectranet,2019-03-02 15:08:26,i can't find my spectranet_ng mifi and i still...,Twitter for iPhone,Negative,[None],[None]
1174,sprectranet,2020-10-30 00:19:29+00:00,spectranet is always terrible at night. fix up...,Twitter for iPhone,Negative,[None],[None]
1175,sprectranet,2020-04-26 07:15:37+00:00,spectranet_ng are we getting 100% today ?,Twitter for iPhone,Neutral,[None],[None]


## 6. Write results to CSVs

In [16]:
spec_data_with_sentiment.to_csv("../data/model-generated/spectranet_full_prediction.csv",index=False)

## 7. Format data for visualization
Here I flatten the lists so that each row has just a single aspect and its corresponding sentiment. As such, there will be rows with duplicate tweets, etc.

In [17]:
def flatten_df(df,main_col,corr_col):
    
    """
    Ungroup the ABSA predictions. Here we flatten lists such that each row only has a single aspect
    and its corresponding sentiment. As such, there will be rows with duplicate tweets
    """
    
    #Empty dataframe to store the flattened dataframe in
    flattened_df = pd.DataFrame()
    
    #Iterate through all the rows in the dataframe
    for idx in range(len(df)):
        
        #Get the current row
        row = df.iloc[idx,:]
        
        #Get the value of the main column (aspect column) to be flattened in the row
        aspect_col = row[main_col]
        
        #If it is [None]
        if aspect_col == [None]:
            
            #Replace the values with np.nan
            row[[main_col,corr_col]] = np.nan
            
            #Add the row to the dataframe
            flattened_df = flattened_df.append(row)
            
        else:
            
            #Get the number of predicted aspects
            num_aspects = len(aspect_col)
            
            #Iterate through all the predicted aspects and their corresponding sentiments
            for i in range(num_aspects):
                
                #Make a copy of the row for editing
                row_to_edit = row.copy()
                
                #Get an aspect and its corresponding sentiment 
                row_to_edit[[main_col,corr_col]] = aspect_col[i], row[corr_col][i]
                
                #Add the flattened row to the dataframe
                flattened_df = flattened_df.append(row_to_edit)
    
    return flattened_df
                

In [18]:
spectranet_flattened = flatten_df(spec_data_with_sentiment,'Detected aspects','Aspect sentiment')

/Users/koredeakande/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/koredeakande/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()
/Users/koredeakande/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [19]:
spectranet_flattened

,ISP_Name,Time,Text,Source,Predicted sentiment,Detected aspects,Aspect sentiment
0,sprectranet,2019-05-13 09:30:03,it gives me joy seeing my spectranet turning g...,Twitter for iPhone,Positive,speed,Negative
1,sprectranet,2020-04-21 06:11:55+00:00,spectranet_ng is this even fair? i won't renew...,Twitter for iPhone,Negative,NaN,NaN
2,sprectranet,2020-02-04 18:30:35+00:00,my family used my spectranet and they don't wa...,Twitter for Android,Negative,NaN,NaN
3,sprectranet,2019-02-16 18:11:48,spectranet_ng can i subscribe via ubagroup mob...,Twitter for Android,Neutral,NaN,NaN
4,sprectranet,2020-08-14 06:25:29+00:00,eniolashitta youtube is where spectranet start...,Twitter for Android,Neutral,speed,Negative
...,...,...,...,...,...,...,...
1172,sprectranet,2019-01-27 07:45:17,riqueza_cakes get spectranet then .,Twitter for iPhone,Neutral,NaN,NaN
1173,sprectranet,2019-03-02 15:08:26,i can't find my spectranet_ng mifi and i still...,Twitter for iPhone,Negative,NaN,NaN
1174,sprectranet,2020-10-30 00:19:29+00:00,spectranet is always terrible at night. fix up...,Twitter for iPhone,Negative,NaN,NaN
1175,sprectranet,2020-04-26 07:15:37+00:00,spectranet_ng are we getting 100% today ?,Twitter for iPhone,Neutral,NaN,NaN


## 8. Export flattened dataframe to CSV

In [20]:
spectranet_flattened.to_csv("../data/model-generated/spectranet_predictions_flattened.csv",index=False)

In [21]:
spectranet_flattened.groupby('Predicted sentiment')['Text'].nunique()

Predicted sentiment
Negative    577
Neutral     491
Positive    104
Name: Text, dtype: int64